In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk

In [ ]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
config = backend.configuration()
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
beta = 3 * np.pi / 4
gamma = np.pi / 2

In [ ]:
schedules = []

In [ ]:
H = cmd_def.get('u2', qubits=[0], P0=0, P1=np.pi) | cmd_def.get('u2', qubits=[1], P0=0, P1=np.pi) | cmd_def.get('u2', qubits=[2], P0=0, P1=np.pi) | cmd_def.get('u2', qubits=[3], P0=0, P1=np.pi) | cmd_def.get('u2', qubits=[4], P0=0, P1=np.pi)

CNOT_RZ_CNOT_01 = cmd_def.get('cx', qubits=[0, 1])
CNOT_RZ_CNOT_01 |= cmd_def.get('u1', qubits=[1], P0=gamma) << CNOT_RZ_CNOT_01.duration
CNOT_RZ_CNOT_01 |= cmd_def.get('cx', qubits=[0, 1]) << CNOT_RZ_CNOT_01.duration

CNOT_RZ_CNOT_12 = cmd_def.get('cx', qubits=[1, 2])
CNOT_RZ_CNOT_12 |= cmd_def.get('u1', qubits=[2], P0=gamma) << CNOT_RZ_CNOT_12.duration
CNOT_RZ_CNOT_12 |= cmd_def.get('cx', qubits=[1, 2]) << CNOT_RZ_CNOT_12.duration

CNOT_RZ_CNOT_23 = cmd_def.get('cx', qubits=[2, 3])
CNOT_RZ_CNOT_23 |= cmd_def.get('u1', qubits=[3], P0=gamma) << CNOT_RZ_CNOT_23.duration
CNOT_RZ_CNOT_23 |= cmd_def.get('cx', qubits=[2, 3]) << CNOT_RZ_CNOT_23.duration

CNOT_RZ_CNOT_34 = cmd_def.get('cx', qubits=[3, 4])
CNOT_RZ_CNOT_34 |= cmd_def.get('u1', qubits=[4], P0=gamma) << CNOT_RZ_CNOT_34.duration
CNOT_RZ_CNOT_34 |= cmd_def.get('cx', qubits=[3, 4]) << CNOT_RZ_CNOT_34.duration

# RX(beta) = U3(beta, 3*pi/2, pi/2), hence line below
RX = cmd_def.get('u3', qubits=[0], P0=beta, P1=3*np.pi/2, P2=np.pi/2) | cmd_def.get('u3', qubits=[1], P0=beta, P1=3*np.pi/2, P2=np.pi/2) | cmd_def.get('u3', qubits=[2], P0=beta, P1=3*np.pi/2, P2=np.pi/2) | cmd_def.get('u3', qubits=[3], P0=beta, P1=3*np.pi/2, P2=np.pi/2) | cmd_def.get('u3', qubits=[4], P0=beta, P1=3*np.pi/2, P2=np.pi/2)

schedule = H
schedule |= CNOT_RZ_CNOT_01 << schedule.duration
schedule |= CNOT_RZ_CNOT_12 << schedule.duration
schedule |= CNOT_RZ_CNOT_23 << schedule.duration
schedule |= CNOT_RZ_CNOT_34 << schedule.duration
schedule |= RX << schedule.duration
schedule |= measure << schedule.duration

schedules.append(schedule)

In [ ]:
## Optimized Way: zz_interaction decomposes directly to cross resonancefor theta in thetas:
circuit = q.QuantumCircuit(5)
circuit.h(0)
circuit.h(1)
circuit.h(2)
circuit.h(3)
circuit.h(4)

circuit.zz_interaction(gamma, 0, 1)
circuit.zz_interaction(gamma, 1, 2)
circuit.zz_interaction(gamma, 2, 3)
circuit.zz_interaction(gamma, 3, 4)

circuit.rx(beta, 0)
circuit.rx(beta, 1)
circuit.rx(beta, 2)
circuit.rx(beta, 3)
circuit.rx(beta, 4)

decomposed_circuit = circuit.decompose()

pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
schedule |= measure << schedule.duration
schedules.append(schedule)

In [ ]:
shots = 8000
job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % job.job_id())

In [ ]:
job.status()

# Run a mitigation cal job

In [ ]:
import qiskit.ignis.mitigation.measurement as mit
meas_qcs, meas_labels = mit.complete_meas_cal(qubit_list=[0,1,2,3,4])
meas_qcs_transpiled = q.transpile(meas_qcs, backend, basis_gates=['x'])
meas_schedules = q.schedule(meas_qcs_transpiled, backend=backend, cmd_def=cmd_def)
job_mit = q.execute(meas_schedules, backend)

In [ ]:
job_mit.status()

-----
## Analysis

In [ ]:
from qiskit.result import marginal_counts

In [ ]:
result = job.result()

In [ ]:
# mitigate results
result_mit = marginal_counts(job_mit.result(), [0,1,2,3,4])
readout_fitter = mit.CompleteMeasFitter(result_mit, meas_labels)
readout_fitter.cal_matrix

In [ ]:
standard_counts = marginal_counts(result.get_counts(0), [0,1,2,3,4])
print('before mitigation:')
print(standard_counts)

standard_counts = readout_fitter.filter.apply(standard_counts)
print('\n\n\n after mitigation:')
print(standard_counts)

In [ ]:
optimized_counts = marginal_counts(result.get_counts(1), [0,1,2,3,4])
print('before mitigation:')
print(optimized_counts)

optimized_counts = readout_fitter.filter.apply(optimized_counts)
print('\n\n\n after mitigation:')
print(optimized_counts)

In [ ]:
# sanitize data and plot
for key in standard_counts:
    standard_counts[key] = int(standard_counts[key])
for key in optimized_counts:
    optimized_counts[key] = int(optimized_counts[key])

In [ ]:
standard_counts

In [ ]:
optimized_counts

In [ ]:
plot_histogram([standard_counts, optimized_counts], legend=['standard', 'optimized'])

In [ ]:
# ideal_counts = {'01': 4000, '10': 4000}
# standard_counts = {'00': 9.306905529044516e-10, '01': 4340.499596429236, '10': 3659.5004035698817}
# optimized_counts = {'00': 3.2117147307346117e-09, '01': 4201.575210406215, '10': 3798.4247895903336, '11': 2.401224165590816e-10}

In [ ]:
# print(pulse_compiler_helper_fns.kl_divergence(ideal_counts, standard_counts))
# print(pulse_compiler_helper_fns.kl_divergence(ideal_counts, optimized_counts))

In [ ]:
# print(pulse_compiler_helper_fns.cross_entropy(ideal_counts, standard_counts))
# print(pulse_compiler_helper_fns.cross_entropy(ideal_counts, optimized_counts))